### **Curating bcc_face_nose1.h5ad**

Article:  Multi-scale spatial mapping of cell populations across anatomical sites in healthy human skin and basal cell carcinoma

DOI: https://doi.org/10.1101/2023.08.08.551504

Data Source : https://spatial-skin-atlas.cellgeni.sanger.ac.uk

##### **Mount farm**

mount-farm

##### **Packages required for curation**

In [1]:
#Import all packages required for curation

In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import scipy
from tqdm import tqdm
from scipy import sparse
from scipy.sparse import csr_matrix
import anndata as ad
import os
import subprocess
import math

### **Curation Schema**

##### **X (Matrix Layers)**

##### **AnnData object**

In [3]:
# Load the AnnData object

In [4]:
adata = sc.read_h5ad('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/spatial-skin/Data/WSSKNKCLsp12140273.h5ad')

In [5]:
# View the AnnData object

In [6]:
adata

AnnData object with n_obs × n_vars = 1200 × 19614
    obs: 'in_tissue', 'array_row', 'array_col', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes', 'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc', 'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes', 'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts', 'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC', 'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes', 'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1'
    uns: 'leiden', 'log1p', 'neighbors', 'pca', 'spatial', 'umap'
    obsm: 'X_spatial'

##### **X- expression matrix**

In [7]:
# View the expression matrix of the anndata object

In [8]:
adata.X

array([[-0.0810534 , -0.08124188, -0.09260149, ..., -0.12655571,
         1.1178309 , -0.05735825],
       [-0.0810534 , -0.08124188, -0.09260149, ..., -0.12655571,
        -0.26154166, -0.05735825],
       [-0.0810534 , -0.08124188, -0.09260149, ..., -0.12655571,
        -0.26154166, -0.05735825],
       ...,
       [-0.0810534 , -0.08124188, -0.09260149, ..., -0.12655571,
        -0.26154166, -0.05735825],
       [-0.0810534 , -0.08124188, -0.09260149, ..., -0.12655571,
        -0.26154166, -0.05735825],
       [-0.0810534 , -0.08124188, -0.09260149, ..., -0.12655571,
        -0.26154166, -0.05735825]], dtype=float32)

In [9]:
#data type of adata.X

In [10]:
type(adata.X)

numpy.ndarray

In [11]:
#convert to csr_matrix

In [12]:
adata.X = csr_matrix(adata.X)

In [13]:
adata.X

<1200x19614 sparse matrix of type '<class 'numpy.float32'>'
	with 23536800 stored elements in Compressed Sparse Row format>

In [14]:
# Print the matrix to check whether they are normalized counts or raw counts. if the matrix has floating numbers,they are normalized counts.if they are integers, they are raw counts.

In [15]:
print(adata.X)

  (0, 0)	-0.0810534
  (0, 1)	-0.081241876
  (0, 2)	-0.09260149
  (0, 3)	-0.28672016
  (0, 4)	-0.059513275
  (0, 5)	-0.4246031
  (0, 6)	-0.1061563
  (0, 7)	-0.1974151
  (0, 8)	-0.22630191
  (0, 9)	-0.53164613
  (0, 10)	0.7781726
  (0, 11)	-0.42148858
  (0, 12)	-0.39124823
  (0, 13)	-0.06684169
  (0, 14)	-0.24676105
  (0, 15)	0.85745966
  (0, 16)	-0.418202
  (0, 17)	1.5444577
  (0, 18)	-0.19792707
  (0, 19)	-0.0011533205
  (0, 20)	-0.38426527
  (0, 21)	-0.04793396
  (0, 22)	0.19897756
  (0, 23)	-0.44924402
  (0, 24)	0.08223743
  :	:
  (1199, 19589)	0.8790614
  (1199, 19590)	-0.0913108
  (1199, 19591)	0.838721
  (1199, 19592)	-0.4623503
  (1199, 19593)	-0.38929185
  (1199, 19594)	0.07567622
  (1199, 19595)	-0.32647318
  (1199, 19596)	0.8904027
  (1199, 19597)	-0.33694646
  (1199, 19598)	-0.1548765
  (1199, 19599)	-0.41850504
  (1199, 19600)	-0.9054277
  (1199, 19601)	0.2723437
  (1199, 19602)	0.49004725
  (1199, 19603)	-0.16283216
  (1199, 19604)	-0.7083904
  (1199, 19605)	-0.13756171
  (

##### **Raw counts matrix**

In [16]:
# If X has normalized counts, check for the raw counts matrix.

In [17]:
#Here the raw counts are provided in a separate object, load the raw counts matrix

In [18]:
araw= sc.read_10x_h5('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/spatial-skin/Raw/all/WSSKNKCLsp12140273/filtered_feature_bc_matrix.h5')

/nfs/users/nfs_s/sa32/anaconda/envs/shibla_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [19]:
# view raw object

In [20]:
araw

AnnData object with n_obs × n_vars = 1200 × 36601
    var: 'gene_ids', 'feature_types', 'genome'

In [21]:
# view raw matrix

In [22]:
araw.X

<1200x36601 sparse matrix of type '<class 'numpy.float32'>'
	with 5564066 stored elements in Compressed Sparse Row format>

In [23]:
print(araw.X)

  (0, 24)	4.0
  (0, 29)	6.0
  (0, 30)	1.0
  (0, 32)	5.0
  (0, 36)	1.0
  (0, 42)	1.0
  (0, 44)	6.0
  (0, 45)	1.0
  (0, 50)	1.0
  (0, 51)	7.0
  (0, 52)	1.0
  (0, 53)	2.0
  (0, 57)	3.0
  (0, 58)	1.0
  (0, 59)	13.0
  (0, 60)	7.0
  (0, 62)	4.0
  (0, 70)	3.0
  (0, 71)	4.0
  (0, 73)	8.0
  (0, 75)	5.0
  (0, 77)	2.0
  (0, 78)	6.0
  (0, 84)	1.0
  (0, 85)	2.0
  :	:
  (1199, 36407)	3.0
  (1199, 36410)	1.0
  (1199, 36412)	1.0
  (1199, 36413)	3.0
  (1199, 36415)	2.0
  (1199, 36423)	1.0
  (1199, 36429)	2.0
  (1199, 36432)	1.0
  (1199, 36443)	1.0
  (1199, 36450)	6.0
  (1199, 36492)	1.0
  (1199, 36513)	2.0
  (1199, 36516)	1.0
  (1199, 36519)	1.0
  (1199, 36559)	21.0
  (1199, 36560)	10.0
  (1199, 36561)	36.0
  (1199, 36562)	43.0
  (1199, 36563)	1.0
  (1199, 36564)	23.0
  (1199, 36565)	43.0
  (1199, 36566)	19.0
  (1199, 36568)	37.0
  (1199, 36569)	4.0
  (1199, 36571)	12.0


In [24]:
# since the raw object is combined one, extract the raw counts for this dataset 

##### **Variables(var)**

In [25]:
#View the var of anndata and raw object

In [26]:
adata.var

""
AL627309.1
AL627309.5
AP006222.2
LINC01409
FAM87B
...
AC240274.1
AC233755.2
AC233755.1
AC007325.4


In [27]:
araw.var

,gene_ids,feature_types,genome
MIR1302-2HG,ENSG00000243485,Gene Expression,GRCh38
FAM138A,ENSG00000237613,Gene Expression,GRCh38
OR4F5,ENSG00000186092,Gene Expression,GRCh38
AL627309.1,ENSG00000238009,Gene Expression,GRCh38
AL627309.3,ENSG00000239945,Gene Expression,GRCh38
...,...,...,...
AC141272.1,ENSG00000277836,Gene Expression,GRCh38
AC023491.2,ENSG00000278633,Gene Expression,GRCh38
AC007325.1,ENSG00000276017,Gene Expression,GRCh38
AC007325.4,ENSG00000278817,Gene Expression,GRCh38


In [28]:
#Ensembl IDs

In [29]:
gene_info = pd.read_table('/lustre/scratch127/cellgen/cellgeni/shibla/ref_files/2020A.gene_names.tsv')

In [30]:
gene_info

,ensembl_ids,gene,version
0,ENSG00000243485,MIR1302-2HG,2020A
1,ENSG00000237613,FAM138A,2020A
2,ENSG00000186092,OR4F5,2020A
3,ENSG00000238009,AL627309.1,2020A
4,ENSG00000239945,AL627309.3,2020A
...,...,...,...
36596,ENSG00000277836,AC141272.1,2020A
36597,ENSG00000278633,AC023491.2,2020A
36598,ENSG00000276017,AC007325.1,2020A
36599,ENSG00000278817,AC007325.4,2020A


In [31]:
#create a dictionary with gene symbols and ensembl ids from the gene information file

In [32]:
gene_info_genesym_to_ensembl = dict(zip(gene_info['gene'],gene_info['ensembl_ids']))

In [33]:
gene_info_genesym_to_ensembl

{'MIR1302-2HG': 'ENSG00000243485',
 'FAM138A': 'ENSG00000237613',
 'OR4F5': 'ENSG00000186092',
 'AL627309.1': 'ENSG00000238009',
 'AL627309.3': 'ENSG00000239945',
 'AL627309.2': 'ENSG00000239906',
 'AL627309.5': 'ENSG00000241860',
 'AL627309.4': 'ENSG00000241599',
 'AP006222.2': 'ENSG00000286448',
 'AL732372.1': 'ENSG00000236601',
 'OR4F29': 'ENSG00000284733',
 'AC114498.1': 'ENSG00000235146',
 'OR4F16': 'ENSG00000284662',
 'AL669831.2': 'ENSG00000229905',
 'LINC01409': 'ENSG00000237491',
 'FAM87B': 'ENSG00000177757',
 'LINC01128': 'ENSG00000228794',
 'LINC00115': 'ENSG00000225880',
 'FAM41C': 'ENSG00000230368',
 'AL645608.6': 'ENSG00000272438',
 'AL645608.2': 'ENSG00000230699',
 'AL645608.4': 'ENSG00000241180',
 'LINC02593': 'ENSG00000223764',
 'SAMD11': 'ENSG00000187634',
 'NOC2L': 'ENSG00000188976',
 'KLHL17': 'ENSG00000187961',
 'PLEKHN1': 'ENSG00000187583',
 'PERM1': 'ENSG00000187642',
 'AL645608.7': 'ENSG00000272512',
 'HES4': 'ENSG00000188290',
 'ISG15': 'ENSG00000187608',
 'AL6

In [34]:
#Store ensembl ids in a new column in adata.var by matching gene symbols and ensembl ids from the gene information file

In [35]:
adata.var['gene_symbols'] = adata.var_names

In [36]:
araw.var['gene_symbols'] = araw.var_names

In [37]:
araw.var_names = araw.var['gene_ids']

In [38]:
adata.var['ensembl_id'] = adata.var['gene_symbols'].map(gene_info_genesym_to_ensembl)

In [39]:
adata.var

,gene_symbols,ensembl_id
AL627309.1,AL627309.1,ENSG00000238009
AL627309.5,AL627309.5,ENSG00000241860
AP006222.2,AP006222.2,ENSG00000286448
LINC01409,LINC01409,ENSG00000237491
FAM87B,FAM87B,ENSG00000177757
...,...,...
AC240274.1,AC240274.1,ENSG00000271254
AC233755.2,AC233755.2,ENSG00000277856
AC233755.1,AC233755.1,ENSG00000275063
AC007325.4,AC007325.4,ENSG00000278817


In [40]:
adata.var['ensembl_id'].isna().sum()

16

In [41]:
adata.var_names = adata.var['ensembl_id']

In [42]:
adata.var

,gene_symbols,ensembl_id
ensembl_id,,
ENSG00000238009,AL627309.1,ENSG00000238009
ENSG00000241860,AL627309.5,ENSG00000241860
ENSG00000286448,AP006222.2,ENSG00000286448
ENSG00000237491,LINC01409,ENSG00000237491
ENSG00000177757,FAM87B,ENSG00000177757
...,...,...
ENSG00000271254,AC240274.1,ENSG00000271254
ENSG00000277856,AC233755.2,ENSG00000277856
ENSG00000275063,AC233755.1,ENSG00000275063


In [43]:
del adata.var['gene_symbols']
del adata.var['ensembl_id']

In [44]:
del araw.var['gene_ids']
del araw.var['feature_types']
del araw.var['genome']
del araw.var['gene_symbols']

In [45]:
adata.var.index= adata.var.index.drop_duplicates()

In [46]:
adata.var

""
ensembl_id
ENSG00000238009
ENSG00000241860
ENSG00000286448
ENSG00000237491
ENSG00000177757
...
ENSG00000271254
ENSG00000277856
ENSG00000275063


In [47]:
araw.var

""
gene_ids
ENSG00000243485
ENSG00000237613
ENSG00000186092
ENSG00000238009
ENSG00000239945
...
ENSG00000277836
ENSG00000278633
ENSG00000276017


In [48]:
# load the approved genes file

In [49]:
approved_genes = pd.read_csv('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/Endometrium_reference_integrated_atlas/genes_approved.csv')

In [50]:
# Create a dictionary from the approved genes file using the symbols and feature id columns.

In [51]:
genedict = {key: 1 for key in list(approved_genes.feature_id)}

In [52]:
genedict

{'ERCC-00002': 1,
 'ERCC-00003': 1,
 'ERCC-00004': 1,
 'ERCC-00009': 1,
 'ERCC-00012': 1,
 'ERCC-00013': 1,
 'ERCC-00014': 1,
 'ERCC-00016': 1,
 'ERCC-00017': 1,
 'ERCC-00019': 1,
 'ERCC-00022': 1,
 'ERCC-00024': 1,
 'ERCC-00025': 1,
 'ERCC-00028': 1,
 'ERCC-00031': 1,
 'ERCC-00033': 1,
 'ERCC-00034': 1,
 'ERCC-00035': 1,
 'ERCC-00039': 1,
 'ERCC-00040': 1,
 'ERCC-00041': 1,
 'ERCC-00042': 1,
 'ERCC-00043': 1,
 'ERCC-00044': 1,
 'ERCC-00046': 1,
 'ERCC-00048': 1,
 'ERCC-00051': 1,
 'ERCC-00053': 1,
 'ERCC-00054': 1,
 'ERCC-00057': 1,
 'ERCC-00058': 1,
 'ERCC-00059': 1,
 'ERCC-00060': 1,
 'ERCC-00061': 1,
 'ERCC-00062': 1,
 'ERCC-00067': 1,
 'ERCC-00069': 1,
 'ERCC-00071': 1,
 'ERCC-00073': 1,
 'ERCC-00074': 1,
 'ERCC-00075': 1,
 'ERCC-00076': 1,
 'ERCC-00077': 1,
 'ERCC-00078': 1,
 'ERCC-00079': 1,
 'ERCC-00081': 1,
 'ERCC-00083': 1,
 'ERCC-00084': 1,
 'ERCC-00085': 1,
 'ERCC-00086': 1,
 'ERCC-00092': 1,
 'ERCC-00095': 1,
 'ERCC-00096': 1,
 'ERCC-00097': 1,
 'ERCC-00098': 1,
 'ERCC-000

In [53]:
len(genedict)

119799

In [54]:
# Filter out the genes which are not in the approved genes file

In [55]:
var_to_keep_adata = [x for x in adata.var_names if (x in genedict)]
var_to_keep_araw = [x for x in araw.var_names if (x in genedict)]

In [56]:
len(var_to_keep_adata)

19514

In [57]:
len(var_to_keep_araw)

36390

In [58]:
# Modify the anndata object by filtering out the filtered genes. copy the index column values to a new column called gene_symbols

In [59]:
adata = adata[:, var_to_keep_adata].copy()
araw = araw[:, var_to_keep_araw].copy()

In [60]:
#  View the var

In [61]:
adata.var

""
ensembl_id
ENSG00000238009
ENSG00000241860
ENSG00000286448
ENSG00000237491
ENSG00000177757
...
ENSG00000271254
ENSG00000277856
ENSG00000275063


In [62]:
araw.var

""
gene_ids
ENSG00000243485
ENSG00000237613
ENSG00000186092
ENSG00000238009
ENSG00000239945
...
ENSG00000277836
ENSG00000278633
ENSG00000276017


feature is filtered

In [63]:
def add_zero():
	global adata
	global araw
	if araw.shape[1] > adata.shape[1]:
		genes_add = [x for x in araw.var.index.to_list() if x not in adata.var.index.to_list()]
		new_matrix = sparse.csr_matrix((adata.X.data, adata.X.indices, adata.X.indptr), shape = araw.shape)
		all_genes = adata.var.index.to_list()
		all_genes.extend(genes_add)
		new_var = pd.DataFrame(index=all_genes)
		new_var = pd.merge(new_var, araw.var, left_index=True, right_index=True, how='left')
		new_var['feature_is_filtered'] = False
		new_var.loc[genes_add, 'feature_is_filtered'] = True
		new_adata = ad.AnnData(X=new_matrix, obs=adata.obs, var=new_var, uns=adata.uns, obsm=adata.obsm)
		if adata.layers:
			for layer in adata.layers:
				new_layer = sparse.csr_matrix((adata.layers[layer].data, adata.layers[layer].indices, adata.layers[layer].indptr), shape = araw.shape)
				new_adata.layers[layer] = new_layer
		new_adata = new_adata[:,araw.var.index.to_list()]
		new_adata.var = new_adata.var.merge(adata.var, left_index=True, right_index=True, how='left')
		adata = new_adata
	else:
		adata.var['feature_is_filtered'] = False

In [64]:
add_zero()

In [65]:
# view var

In [66]:
adata.var

,feature_is_filtered
ENSG00000243485,True
ENSG00000237613,True
ENSG00000186092,True
ENSG00000238009,False
ENSG00000239945,True
...,...
ENSG00000277836,True
ENSG00000278633,True
ENSG00000276017,True
ENSG00000278817,False


In [67]:
list(adata.var['feature_is_filtered'].unique())

[True, False]

In [68]:
True_count = (adata.var['feature_is_filtered']== True).sum()

In [69]:
True_count

16876

In [70]:
araw.var

""
gene_ids
ENSG00000243485
ENSG00000237613
ENSG00000186092
ENSG00000238009
ENSG00000239945
...
ENSG00000277836
ENSG00000278633
ENSG00000276017


#### **Observations(obs) (Cell metadata)**

In [71]:
#view obs

In [72]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_PTGDS+ fibroblasts,c2l_MigDC,c2l_Neuronal_SchwannC,c2l_SMC,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,8420,54784,1315,2.400336,13115,23.939470,AAACAGCTTTCAGAAG-1,...,0.000101,0.000859,0.000080,0.000008,0.755642,8,15.491590,2.781287,0.000578,0.000082
AAACCGGGTAGGTACC-1,1,42,28,1955,4347,44,1.012192,443,10.190936,AAACCGGGTAGGTACC-1,...,0.021254,0.290084,0.009220,0.156010,0.020516,6,1.386501,0.048015,0.002816,0.015143
AAACCTCATGAAGTTG-1,1,37,19,8114,47734,590,1.236016,10006,20.961998,AAACCTCATGAAGTTG-1,...,0.033619,0.000755,0.006920,0.001462,0.344813,6,5.881720,3.950605,0.060104,0.000450
AAACGAGACGGTTGAT-1,1,35,79,6880,23861,617,2.585809,4262,17.861784,AAACGAGACGGTTGAT-1,...,0.001830,0.000196,0.002325,0.000077,0.000060,7,0.259800,8.589365,0.020901,0.000063
AAACTTGCAAACGTAT-1,1,45,19,7110,30728,764,2.486332,4286,13.948191,AAACTTGCAAACGTAT-1,...,0.041668,0.002979,0.012167,0.000224,0.201390,6,3.901977,0.138767,0.000121,0.164657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTAATCCGTACTCG-1,1,35,55,2856,10697,104,0.972235,949,8.871646,TTGTAATCCGTACTCG-1,...,0.038994,0.066201,0.018657,0.004831,0.006133,3,0.680920,0.057632,0.002339,0.057504
TTGTGAGGCATGACGC-1,1,43,13,5261,16840,185,1.098575,2632,15.629454,TTGTGAGGCATGACGC-1,...,0.262246,0.005857,0.005621,0.000177,0.252717,6,3.618451,1.020156,0.000635,0.020752
TTGTGTTTCCCGAAAG-1,1,51,59,5797,25722,242,0.940829,2508,9.750408,TTGTGTTTCCCGAAAG-1,...,0.134413,0.947343,0.003457,0.000016,0.001602,4,0.501840,0.010743,0.000977,0.027716


In [73]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1'],
      dtype='object')

#### **assay_ontology_term_id**

In [74]:
# add the assay_ontology_term_id column

In [75]:
adata.obs['assay_ontology_term_id'] = ['EFO:0010961'] * len(adata.obs)

In [76]:
# change datatype of the column

In [77]:
adata.obs['assay_ontology_term_id'] = adata.obs['assay_ontology_term_id'].astype('category')

In [78]:
# view adata.obs

In [79]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_MigDC,c2l_Neuronal_SchwannC,c2l_SMC,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,8420,54784,1315,2.400336,13115,23.939470,AAACAGCTTTCAGAAG-1,...,0.000859,0.000080,0.000008,0.755642,8,15.491590,2.781287,0.000578,0.000082,EFO:0010961
AAACCGGGTAGGTACC-1,1,42,28,1955,4347,44,1.012192,443,10.190936,AAACCGGGTAGGTACC-1,...,0.290084,0.009220,0.156010,0.020516,6,1.386501,0.048015,0.002816,0.015143,EFO:0010961
AAACCTCATGAAGTTG-1,1,37,19,8114,47734,590,1.236016,10006,20.961998,AAACCTCATGAAGTTG-1,...,0.000755,0.006920,0.001462,0.344813,6,5.881720,3.950605,0.060104,0.000450,EFO:0010961
AAACGAGACGGTTGAT-1,1,35,79,6880,23861,617,2.585809,4262,17.861784,AAACGAGACGGTTGAT-1,...,0.000196,0.002325,0.000077,0.000060,7,0.259800,8.589365,0.020901,0.000063,EFO:0010961
AAACTTGCAAACGTAT-1,1,45,19,7110,30728,764,2.486332,4286,13.948191,AAACTTGCAAACGTAT-1,...,0.002979,0.012167,0.000224,0.201390,6,3.901977,0.138767,0.000121,0.164657,EFO:0010961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTAATCCGTACTCG-1,1,35,55,2856,10697,104,0.972235,949,8.871646,TTGTAATCCGTACTCG-1,...,0.066201,0.018657,0.004831,0.006133,3,0.680920,0.057632,0.002339,0.057504,EFO:0010961
TTGTGAGGCATGACGC-1,1,43,13,5261,16840,185,1.098575,2632,15.629454,TTGTGAGGCATGACGC-1,...,0.005857,0.005621,0.000177,0.252717,6,3.618451,1.020156,0.000635,0.020752,EFO:0010961
TTGTGTTTCCCGAAAG-1,1,51,59,5797,25722,242,0.940829,2508,9.750408,TTGTGTTTCCCGAAAG-1,...,0.947343,0.003457,0.000016,0.001602,4,0.501840,0.010743,0.000977,0.027716,EFO:0010961


#### **cell_type_ontology_term_id**

In [80]:
#get the column in adata.obs related. to cell type annotation

In [81]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id'],
      dtype='object')

In [82]:
adata.obsm

AxisArrays with keys: X_spatial

In [83]:
c2l_columns = [col for col in adata.obs.columns if col.startswith('c2l')]

In [84]:
c2l_columns

['c2l_Th',
 'c2l_NK',
 'c2l_APOD+ fibroblasts',
 'c2l_CD8+ T RM',
 'c2l_T reg',
 'c2l_Macro1_2',
 'c2l_DC1',
 'c2l_SFRP2+ fibroblasts',
 'c2l_TAGLN+ pericytes',
 'c2l_POSTN+ fibroblasts',
 'c2l_RGS5+ pericytes',
 'c2l_VEC',
 'c2l_Tc',
 'c2l_ILC_NK',
 'c2l_BC',
 'c2l_Monocytes',
 'c2l_MastC',
 'c2l_Melanocytes',
 'c2l_DC2',
 'c2l_LEC',
 'c2l_PlasmaC',
 'c2l_PTGDS+ fibroblasts',
 'c2l_MigDC',
 'c2l_Neuronal_SchwannC',
 'c2l_SMC',
 'c2l_Skeletal muscle cells',
 'c2l_Suprabasal keratinocytes',
 'c2l_Basal keratinocytes',
 'c2l_Chondrocytes',
 'c2l_IL8+ DC1']

In [85]:
adata.obs['max_c2l_column'] = adata.obs[c2l_columns].idxmax(axis=1)

In [86]:
adata.obs['max_c2l_column_value'] = adata.obs[c2l_columns].max(axis=1)

In [87]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_SMC,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,8420,54784,1315,2.400336,13115,23.939470,AAACAGCTTTCAGAAG-1,...,0.000008,0.755642,8,15.491590,2.781287,0.000578,0.000082,EFO:0010961,c2l_Suprabasal keratinocytes,15.491590
AAACCGGGTAGGTACC-1,1,42,28,1955,4347,44,1.012192,443,10.190936,AAACCGGGTAGGTACC-1,...,0.156010,0.020516,6,1.386501,0.048015,0.002816,0.015143,EFO:0010961,c2l_Suprabasal keratinocytes,1.386501
AAACCTCATGAAGTTG-1,1,37,19,8114,47734,590,1.236016,10006,20.961998,AAACCTCATGAAGTTG-1,...,0.001462,0.344813,6,5.881720,3.950605,0.060104,0.000450,EFO:0010961,c2l_Suprabasal keratinocytes,5.881720
AAACGAGACGGTTGAT-1,1,35,79,6880,23861,617,2.585809,4262,17.861784,AAACGAGACGGTTGAT-1,...,0.000077,0.000060,7,0.259800,8.589365,0.020901,0.000063,EFO:0010961,c2l_Basal keratinocytes,8.589365
AAACTTGCAAACGTAT-1,1,45,19,7110,30728,764,2.486332,4286,13.948191,AAACTTGCAAACGTAT-1,...,0.000224,0.201390,6,3.901977,0.138767,0.000121,0.164657,EFO:0010961,c2l_Suprabasal keratinocytes,3.901977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTAATCCGTACTCG-1,1,35,55,2856,10697,104,0.972235,949,8.871646,TTGTAATCCGTACTCG-1,...,0.004831,0.006133,3,0.680920,0.057632,0.002339,0.057504,EFO:0010961,c2l_PlasmaC,2.054078
TTGTGAGGCATGACGC-1,1,43,13,5261,16840,185,1.098575,2632,15.629454,TTGTGAGGCATGACGC-1,...,0.000177,0.252717,6,3.618451,1.020156,0.000635,0.020752,EFO:0010961,c2l_Suprabasal keratinocytes,3.618451
TTGTGTTTCCCGAAAG-1,1,51,59,5797,25722,242,0.940829,2508,9.750408,TTGTGTTTCCCGAAAG-1,...,0.000016,0.001602,4,0.501840,0.010743,0.000977,0.027716,EFO:0010961,c2l_T reg,1.499902


In [88]:
mapping= {'c2l_Th' :'CL:0000912',
 'c2l_NK':'CL:0000623',
 'c2l_APOD+ fibroblasts':'CL:0000057',
 'c2l_CD8+ T RM':'CL:0001203' ,
 'c2l_T reg':'CL:0000815',
 'c2l_Macro1_2':'CL:0000235',
 'c2l_DC1':'CL:0000990',
 'c2l_SFRP2+ fibroblasts':'CL:0000057',
 'c2l_TAGLN+ pericytes':'CL:0000669',
 'c2l_POSTN+ fibroblasts':'CL:0000057',
 'c2l_RGS5+ pericytes':'CL:0000669',
 'c2l_VEC':'CL:0002139',
 'c2l_Tc':'CL:0000910',
 'c2l_ILC_NK':'CL:0001065', #not NK
 'c2l_BC':'CL:0000646',
 'c2l_Monocytes':'CL:0000576',
 'c2l_MastC':'CL:0000097',
 'c2l_Melanocytes':'CL:1000458',
 'c2l_DC2':'CL:0000784',
 'c2l_LEC':'CL:0002138',
 'c2l_PlasmaC':'CL:0000786',
 'c2l_PTGDS+ fibroblasts':'CL:0000057',
 'c2l_MigDC':'CL:0000451', #not mig
 'c2l_Neuronal_SchwannC':'CL:0002573',
 'c2l_SMC':'CL:0000192',
 'c2l_Skeletal muscle cells':'CL:0000188',
 'c2l_Suprabasal keratinocytes':'CL:4033013',
 'c2l_Basal keratinocytes':'CL:0002187',
 'c2l_Chondrocytes':'CL:0000138',
 'c2l_IL8+ DC1':'CL:0000990'}

In [89]:
# create a dictionary of cell type and ontology term

In [90]:
# add the cell_type_ontology_term_id column

In [91]:
adata.obs['cell_type_ontology_term_id'] = adata.obs['max_c2l_column'].map(mapping)

In [92]:
# change datatype of the column

In [93]:
adata.obs['cell_type_ontology_term_id'] = adata.obs['cell_type_ontology_term_id'].astype('category')

In [94]:
list(adata.obs['cell_type_ontology_term_id'].unique())

['CL:4033013',
 'CL:0002187',
 'CL:0000057',
 'CL:0000786',
 'CL:0002139',
 'CL:0000646',
 'CL:0000815',
 'CL:0000623',
 'CL:0000669',
 'CL:0000910',
 'CL:0000451',
 'CL:0002138']

In [95]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,8420,54784,1315,2.400336,13115,23.939470,AAACAGCTTTCAGAAG-1,...,0.755642,8,15.491590,2.781287,0.000578,0.000082,EFO:0010961,c2l_Suprabasal keratinocytes,15.491590,CL:4033013
AAACCGGGTAGGTACC-1,1,42,28,1955,4347,44,1.012192,443,10.190936,AAACCGGGTAGGTACC-1,...,0.020516,6,1.386501,0.048015,0.002816,0.015143,EFO:0010961,c2l_Suprabasal keratinocytes,1.386501,CL:4033013
AAACCTCATGAAGTTG-1,1,37,19,8114,47734,590,1.236016,10006,20.961998,AAACCTCATGAAGTTG-1,...,0.344813,6,5.881720,3.950605,0.060104,0.000450,EFO:0010961,c2l_Suprabasal keratinocytes,5.881720,CL:4033013
AAACGAGACGGTTGAT-1,1,35,79,6880,23861,617,2.585809,4262,17.861784,AAACGAGACGGTTGAT-1,...,0.000060,7,0.259800,8.589365,0.020901,0.000063,EFO:0010961,c2l_Basal keratinocytes,8.589365,CL:0002187
AAACTTGCAAACGTAT-1,1,45,19,7110,30728,764,2.486332,4286,13.948191,AAACTTGCAAACGTAT-1,...,0.201390,6,3.901977,0.138767,0.000121,0.164657,EFO:0010961,c2l_Suprabasal keratinocytes,3.901977,CL:4033013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTAATCCGTACTCG-1,1,35,55,2856,10697,104,0.972235,949,8.871646,TTGTAATCCGTACTCG-1,...,0.006133,3,0.680920,0.057632,0.002339,0.057504,EFO:0010961,c2l_PlasmaC,2.054078,CL:0000786
TTGTGAGGCATGACGC-1,1,43,13,5261,16840,185,1.098575,2632,15.629454,TTGTGAGGCATGACGC-1,...,0.252717,6,3.618451,1.020156,0.000635,0.020752,EFO:0010961,c2l_Suprabasal keratinocytes,3.618451,CL:4033013
TTGTGTTTCCCGAAAG-1,1,51,59,5797,25722,242,0.940829,2508,9.750408,TTGTGTTTCCCGAAAG-1,...,0.001602,4,0.501840,0.010743,0.000977,0.027716,EFO:0010961,c2l_T reg,1.499902,CL:0000815


In [96]:
araw.obs

""
AAACAGCTTTCAGAAG-1
AAACCGGGTAGGTACC-1
AAACCTCATGAAGTTG-1
AAACGAGACGGTTGAT-1
AAACTTGCAAACGTAT-1
...
TTGTAATCCGTACTCG-1
TTGTGAGGCATGACGC-1
TTGTGTTTCCCGAAAG-1
TTGTTGTGTGTCAAGA-1


#### **donor_id**

In [97]:
#identify the column in adata.obs which provides donor information

In [98]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'max_c2l_column', 'max_c2l_column_value',
       'cell_type_ontology_term_id'],
      dtype='object')

In [99]:
# add the donor_id column

In [100]:
adata.obs['donor_id'] = ['bcc_face_nose_A'] * len(adata.obs)

In [101]:
# change datatype of the column

In [102]:
adata.obs['donor_id'] = adata.obs['donor_id'].astype('category')

In [103]:
# view unique values of donor_id column

In [104]:
list(adata.obs['donor_id'].unique())

['bcc_face_nose_A']

In [105]:
#view obs

In [106]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,8420,54784,1315,2.400336,13115,23.939470,AAACAGCTTTCAGAAG-1,...,8,15.491590,2.781287,0.000578,0.000082,EFO:0010961,c2l_Suprabasal keratinocytes,15.491590,CL:4033013,bcc_face_nose_A
AAACCGGGTAGGTACC-1,1,42,28,1955,4347,44,1.012192,443,10.190936,AAACCGGGTAGGTACC-1,...,6,1.386501,0.048015,0.002816,0.015143,EFO:0010961,c2l_Suprabasal keratinocytes,1.386501,CL:4033013,bcc_face_nose_A
AAACCTCATGAAGTTG-1,1,37,19,8114,47734,590,1.236016,10006,20.961998,AAACCTCATGAAGTTG-1,...,6,5.881720,3.950605,0.060104,0.000450,EFO:0010961,c2l_Suprabasal keratinocytes,5.881720,CL:4033013,bcc_face_nose_A
AAACGAGACGGTTGAT-1,1,35,79,6880,23861,617,2.585809,4262,17.861784,AAACGAGACGGTTGAT-1,...,7,0.259800,8.589365,0.020901,0.000063,EFO:0010961,c2l_Basal keratinocytes,8.589365,CL:0002187,bcc_face_nose_A
AAACTTGCAAACGTAT-1,1,45,19,7110,30728,764,2.486332,4286,13.948191,AAACTTGCAAACGTAT-1,...,6,3.901977,0.138767,0.000121,0.164657,EFO:0010961,c2l_Suprabasal keratinocytes,3.901977,CL:4033013,bcc_face_nose_A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTAATCCGTACTCG-1,1,35,55,2856,10697,104,0.972235,949,8.871646,TTGTAATCCGTACTCG-1,...,3,0.680920,0.057632,0.002339,0.057504,EFO:0010961,c2l_PlasmaC,2.054078,CL:0000786,bcc_face_nose_A
TTGTGAGGCATGACGC-1,1,43,13,5261,16840,185,1.098575,2632,15.629454,TTGTGAGGCATGACGC-1,...,6,3.618451,1.020156,0.000635,0.020752,EFO:0010961,c2l_Suprabasal keratinocytes,3.618451,CL:4033013,bcc_face_nose_A
TTGTGTTTCCCGAAAG-1,1,51,59,5797,25722,242,0.940829,2508,9.750408,TTGTGTTTCCCGAAAG-1,...,4,0.501840,0.010743,0.000977,0.027716,EFO:0010961,c2l_T reg,1.499902,CL:0000815,bcc_face_nose_A


In [107]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'max_c2l_column', 'max_c2l_column_value',
       'cell_type_ontology_term_id', 'donor_id'],
      dtype='object')

#### **development_stage_ontology_term_id**

In [108]:
# identify the column in adata which corresponds to age

In [109]:
# add the development_stage_ontology_term_id column

In [110]:
adata.obs['development_stage_ontology_term_id'] = ['HsapDv:0000217'] * len(adata.obs)

In [111]:
# change datatype of the column

In [112]:
adata.obs['development_stage_ontology_term_id'] = adata.obs['development_stage_ontology_term_id'].astype('category')

In [113]:
# view unique values of development_stage_ontology_term_id column

In [114]:
list(adata.obs['development_stage_ontology_term_id'].unique())

['HsapDv:0000217']

In [115]:
# view adata.obs

In [116]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,8420,54784,1315,2.400336,13115,23.939470,AAACAGCTTTCAGAAG-1,...,15.491590,2.781287,0.000578,0.000082,EFO:0010961,c2l_Suprabasal keratinocytes,15.491590,CL:4033013,bcc_face_nose_A,HsapDv:0000217
AAACCGGGTAGGTACC-1,1,42,28,1955,4347,44,1.012192,443,10.190936,AAACCGGGTAGGTACC-1,...,1.386501,0.048015,0.002816,0.015143,EFO:0010961,c2l_Suprabasal keratinocytes,1.386501,CL:4033013,bcc_face_nose_A,HsapDv:0000217
AAACCTCATGAAGTTG-1,1,37,19,8114,47734,590,1.236016,10006,20.961998,AAACCTCATGAAGTTG-1,...,5.881720,3.950605,0.060104,0.000450,EFO:0010961,c2l_Suprabasal keratinocytes,5.881720,CL:4033013,bcc_face_nose_A,HsapDv:0000217
AAACGAGACGGTTGAT-1,1,35,79,6880,23861,617,2.585809,4262,17.861784,AAACGAGACGGTTGAT-1,...,0.259800,8.589365,0.020901,0.000063,EFO:0010961,c2l_Basal keratinocytes,8.589365,CL:0002187,bcc_face_nose_A,HsapDv:0000217
AAACTTGCAAACGTAT-1,1,45,19,7110,30728,764,2.486332,4286,13.948191,AAACTTGCAAACGTAT-1,...,3.901977,0.138767,0.000121,0.164657,EFO:0010961,c2l_Suprabasal keratinocytes,3.901977,CL:4033013,bcc_face_nose_A,HsapDv:0000217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTAATCCGTACTCG-1,1,35,55,2856,10697,104,0.972235,949,8.871646,TTGTAATCCGTACTCG-1,...,0.680920,0.057632,0.002339,0.057504,EFO:0010961,c2l_PlasmaC,2.054078,CL:0000786,bcc_face_nose_A,HsapDv:0000217
TTGTGAGGCATGACGC-1,1,43,13,5261,16840,185,1.098575,2632,15.629454,TTGTGAGGCATGACGC-1,...,3.618451,1.020156,0.000635,0.020752,EFO:0010961,c2l_Suprabasal keratinocytes,3.618451,CL:4033013,bcc_face_nose_A,HsapDv:0000217
TTGTGTTTCCCGAAAG-1,1,51,59,5797,25722,242,0.940829,2508,9.750408,TTGTGTTTCCCGAAAG-1,...,0.501840,0.010743,0.000977,0.027716,EFO:0010961,c2l_T reg,1.499902,CL:0000815,bcc_face_nose_A,HsapDv:0000217


#### **disease_ontology_term_id**

In [117]:
# Assign normal since all are healthy patients

In [118]:
# add the disease_ontology_term_id column

In [119]:
adata.obs['disease_ontology_term_id'] = ['MONDO:0020804'] * len(adata.obs)

In [120]:
#change data type of column

In [121]:
adata.obs['disease_ontology_term_id'] = adata.obs['disease_ontology_term_id'].astype('category')

In [122]:
# view obs

In [123]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,8420,54784,1315,2.400336,13115,23.939470,AAACAGCTTTCAGAAG-1,...,2.781287,0.000578,0.000082,EFO:0010961,c2l_Suprabasal keratinocytes,15.491590,CL:4033013,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804
AAACCGGGTAGGTACC-1,1,42,28,1955,4347,44,1.012192,443,10.190936,AAACCGGGTAGGTACC-1,...,0.048015,0.002816,0.015143,EFO:0010961,c2l_Suprabasal keratinocytes,1.386501,CL:4033013,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804
AAACCTCATGAAGTTG-1,1,37,19,8114,47734,590,1.236016,10006,20.961998,AAACCTCATGAAGTTG-1,...,3.950605,0.060104,0.000450,EFO:0010961,c2l_Suprabasal keratinocytes,5.881720,CL:4033013,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804
AAACGAGACGGTTGAT-1,1,35,79,6880,23861,617,2.585809,4262,17.861784,AAACGAGACGGTTGAT-1,...,8.589365,0.020901,0.000063,EFO:0010961,c2l_Basal keratinocytes,8.589365,CL:0002187,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804
AAACTTGCAAACGTAT-1,1,45,19,7110,30728,764,2.486332,4286,13.948191,AAACTTGCAAACGTAT-1,...,0.138767,0.000121,0.164657,EFO:0010961,c2l_Suprabasal keratinocytes,3.901977,CL:4033013,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTAATCCGTACTCG-1,1,35,55,2856,10697,104,0.972235,949,8.871646,TTGTAATCCGTACTCG-1,...,0.057632,0.002339,0.057504,EFO:0010961,c2l_PlasmaC,2.054078,CL:0000786,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804
TTGTGAGGCATGACGC-1,1,43,13,5261,16840,185,1.098575,2632,15.629454,TTGTGAGGCATGACGC-1,...,1.020156,0.000635,0.020752,EFO:0010961,c2l_Suprabasal keratinocytes,3.618451,CL:4033013,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804
TTGTGTTTCCCGAAAG-1,1,51,59,5797,25722,242,0.940829,2508,9.750408,TTGTGTTTCCCGAAAG-1,...,0.010743,0.000977,0.027716,EFO:0010961,c2l_T reg,1.499902,CL:0000815,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804


#### **is_primary_data**

In [124]:
adata.obs['is_primary_data'] = [True] * len(adata.obs)

In [125]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,8420,54784,1315,2.400336,13115,23.939470,AAACAGCTTTCAGAAG-1,...,0.000578,0.000082,EFO:0010961,c2l_Suprabasal keratinocytes,15.491590,CL:4033013,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True
AAACCGGGTAGGTACC-1,1,42,28,1955,4347,44,1.012192,443,10.190936,AAACCGGGTAGGTACC-1,...,0.002816,0.015143,EFO:0010961,c2l_Suprabasal keratinocytes,1.386501,CL:4033013,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True
AAACCTCATGAAGTTG-1,1,37,19,8114,47734,590,1.236016,10006,20.961998,AAACCTCATGAAGTTG-1,...,0.060104,0.000450,EFO:0010961,c2l_Suprabasal keratinocytes,5.881720,CL:4033013,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True
AAACGAGACGGTTGAT-1,1,35,79,6880,23861,617,2.585809,4262,17.861784,AAACGAGACGGTTGAT-1,...,0.020901,0.000063,EFO:0010961,c2l_Basal keratinocytes,8.589365,CL:0002187,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True
AAACTTGCAAACGTAT-1,1,45,19,7110,30728,764,2.486332,4286,13.948191,AAACTTGCAAACGTAT-1,...,0.000121,0.164657,EFO:0010961,c2l_Suprabasal keratinocytes,3.901977,CL:4033013,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTAATCCGTACTCG-1,1,35,55,2856,10697,104,0.972235,949,8.871646,TTGTAATCCGTACTCG-1,...,0.002339,0.057504,EFO:0010961,c2l_PlasmaC,2.054078,CL:0000786,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True
TTGTGAGGCATGACGC-1,1,43,13,5261,16840,185,1.098575,2632,15.629454,TTGTGAGGCATGACGC-1,...,0.000635,0.020752,EFO:0010961,c2l_Suprabasal keratinocytes,3.618451,CL:4033013,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True
TTGTGTTTCCCGAAAG-1,1,51,59,5797,25722,242,0.940829,2508,9.750408,TTGTGTTTCCCGAAAG-1,...,0.000977,0.027716,EFO:0010961,c2l_T reg,1.499902,CL:0000815,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True


In [126]:
#change data type of column

In [127]:
adata.obs['is_primary_data'] = adata.obs['is_primary_data'].astype('bool')

#### **organism_ontology_term_id**

In [128]:
# assign organism id 

In [129]:
adata.obs['organism_ontology_term_id'] = ['NCBITaxon:9606'] * len(adata.obs)

In [130]:
#change data type of column

In [131]:
adata.obs['organism_ontology_term_id'] = adata.obs['organism_ontology_term_id'].astype('category')

In [132]:
# view obs

In [133]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,8420,54784,1315,2.400336,13115,23.939470,AAACAGCTTTCAGAAG-1,...,0.000082,EFO:0010961,c2l_Suprabasal keratinocytes,15.491590,CL:4033013,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606
AAACCGGGTAGGTACC-1,1,42,28,1955,4347,44,1.012192,443,10.190936,AAACCGGGTAGGTACC-1,...,0.015143,EFO:0010961,c2l_Suprabasal keratinocytes,1.386501,CL:4033013,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606
AAACCTCATGAAGTTG-1,1,37,19,8114,47734,590,1.236016,10006,20.961998,AAACCTCATGAAGTTG-1,...,0.000450,EFO:0010961,c2l_Suprabasal keratinocytes,5.881720,CL:4033013,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606
AAACGAGACGGTTGAT-1,1,35,79,6880,23861,617,2.585809,4262,17.861784,AAACGAGACGGTTGAT-1,...,0.000063,EFO:0010961,c2l_Basal keratinocytes,8.589365,CL:0002187,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606
AAACTTGCAAACGTAT-1,1,45,19,7110,30728,764,2.486332,4286,13.948191,AAACTTGCAAACGTAT-1,...,0.164657,EFO:0010961,c2l_Suprabasal keratinocytes,3.901977,CL:4033013,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTAATCCGTACTCG-1,1,35,55,2856,10697,104,0.972235,949,8.871646,TTGTAATCCGTACTCG-1,...,0.057504,EFO:0010961,c2l_PlasmaC,2.054078,CL:0000786,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606
TTGTGAGGCATGACGC-1,1,43,13,5261,16840,185,1.098575,2632,15.629454,TTGTGAGGCATGACGC-1,...,0.020752,EFO:0010961,c2l_Suprabasal keratinocytes,3.618451,CL:4033013,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606
TTGTGTTTCCCGAAAG-1,1,51,59,5797,25722,242,0.940829,2508,9.750408,TTGTGTTTCCCGAAAG-1,...,0.027716,EFO:0010961,c2l_T reg,1.499902,CL:0000815,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606


#### **self_reported_ethnicity_ontology_term_id**

In [134]:
adata.obs['self_reported_ethnicity_ontology_term_id'] = ['unknown'] * len(adata.obs)

In [135]:
# change data type

In [136]:
adata.obs['self_reported_ethnicity_ontology_term_id'] = adata.obs['self_reported_ethnicity_ontology_term_id'].astype('category')

In [137]:
# view obs

In [138]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,8420,54784,1315,2.400336,13115,23.939470,AAACAGCTTTCAGAAG-1,...,EFO:0010961,c2l_Suprabasal keratinocytes,15.491590,CL:4033013,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown
AAACCGGGTAGGTACC-1,1,42,28,1955,4347,44,1.012192,443,10.190936,AAACCGGGTAGGTACC-1,...,EFO:0010961,c2l_Suprabasal keratinocytes,1.386501,CL:4033013,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown
AAACCTCATGAAGTTG-1,1,37,19,8114,47734,590,1.236016,10006,20.961998,AAACCTCATGAAGTTG-1,...,EFO:0010961,c2l_Suprabasal keratinocytes,5.881720,CL:4033013,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown
AAACGAGACGGTTGAT-1,1,35,79,6880,23861,617,2.585809,4262,17.861784,AAACGAGACGGTTGAT-1,...,EFO:0010961,c2l_Basal keratinocytes,8.589365,CL:0002187,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown
AAACTTGCAAACGTAT-1,1,45,19,7110,30728,764,2.486332,4286,13.948191,AAACTTGCAAACGTAT-1,...,EFO:0010961,c2l_Suprabasal keratinocytes,3.901977,CL:4033013,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTAATCCGTACTCG-1,1,35,55,2856,10697,104,0.972235,949,8.871646,TTGTAATCCGTACTCG-1,...,EFO:0010961,c2l_PlasmaC,2.054078,CL:0000786,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown
TTGTGAGGCATGACGC-1,1,43,13,5261,16840,185,1.098575,2632,15.629454,TTGTGAGGCATGACGC-1,...,EFO:0010961,c2l_Suprabasal keratinocytes,3.618451,CL:4033013,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown
TTGTGTTTCCCGAAAG-1,1,51,59,5797,25722,242,0.940829,2508,9.750408,TTGTGTTTCCCGAAAG-1,...,EFO:0010961,c2l_T reg,1.499902,CL:0000815,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown


In [139]:
list(adata.obs['self_reported_ethnicity_ontology_term_id'].unique())

['unknown']

#### **sex_ontology_term_id**

In [140]:
adata.obs['sex_ontology_term_id'] = ['PATO:0000384'] * len(adata.obs)

In [141]:
# change data type

In [142]:
adata.obs['sex_ontology_term_id'] = adata.obs['sex_ontology_term_id'].astype('category')

In [143]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,8420,54784,1315,2.400336,13115,23.939470,AAACAGCTTTCAGAAG-1,...,c2l_Suprabasal keratinocytes,15.491590,CL:4033013,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384
AAACCGGGTAGGTACC-1,1,42,28,1955,4347,44,1.012192,443,10.190936,AAACCGGGTAGGTACC-1,...,c2l_Suprabasal keratinocytes,1.386501,CL:4033013,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384
AAACCTCATGAAGTTG-1,1,37,19,8114,47734,590,1.236016,10006,20.961998,AAACCTCATGAAGTTG-1,...,c2l_Suprabasal keratinocytes,5.881720,CL:4033013,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384
AAACGAGACGGTTGAT-1,1,35,79,6880,23861,617,2.585809,4262,17.861784,AAACGAGACGGTTGAT-1,...,c2l_Basal keratinocytes,8.589365,CL:0002187,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384
AAACTTGCAAACGTAT-1,1,45,19,7110,30728,764,2.486332,4286,13.948191,AAACTTGCAAACGTAT-1,...,c2l_Suprabasal keratinocytes,3.901977,CL:4033013,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTAATCCGTACTCG-1,1,35,55,2856,10697,104,0.972235,949,8.871646,TTGTAATCCGTACTCG-1,...,c2l_PlasmaC,2.054078,CL:0000786,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384
TTGTGAGGCATGACGC-1,1,43,13,5261,16840,185,1.098575,2632,15.629454,TTGTGAGGCATGACGC-1,...,c2l_Suprabasal keratinocytes,3.618451,CL:4033013,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384
TTGTGTTTCCCGAAAG-1,1,51,59,5797,25722,242,0.940829,2508,9.750408,TTGTGTTTCCCGAAAG-1,...,c2l_T reg,1.499902,CL:0000815,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384


#### **suspension_type**

In [144]:
# since visium suspension type is 'na'

In [145]:
adata.obs['suspension_type'] = ['na'] * len(adata.obs)

In [146]:
# change data type

In [147]:
adata.obs['suspension_type'] = adata.obs['suspension_type'].astype('category')

In [148]:
# view obs

In [149]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,8420,54784,1315,2.400336,13115,23.939470,AAACAGCTTTCAGAAG-1,...,15.491590,CL:4033013,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na
AAACCGGGTAGGTACC-1,1,42,28,1955,4347,44,1.012192,443,10.190936,AAACCGGGTAGGTACC-1,...,1.386501,CL:4033013,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na
AAACCTCATGAAGTTG-1,1,37,19,8114,47734,590,1.236016,10006,20.961998,AAACCTCATGAAGTTG-1,...,5.881720,CL:4033013,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na
AAACGAGACGGTTGAT-1,1,35,79,6880,23861,617,2.585809,4262,17.861784,AAACGAGACGGTTGAT-1,...,8.589365,CL:0002187,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na
AAACTTGCAAACGTAT-1,1,45,19,7110,30728,764,2.486332,4286,13.948191,AAACTTGCAAACGTAT-1,...,3.901977,CL:4033013,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTAATCCGTACTCG-1,1,35,55,2856,10697,104,0.972235,949,8.871646,TTGTAATCCGTACTCG-1,...,2.054078,CL:0000786,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na
TTGTGAGGCATGACGC-1,1,43,13,5261,16840,185,1.098575,2632,15.629454,TTGTGAGGCATGACGC-1,...,3.618451,CL:4033013,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na
TTGTGTTTCCCGAAAG-1,1,51,59,5797,25722,242,0.940829,2508,9.750408,TTGTGTTTCCCGAAAG-1,...,1.499902,CL:0000815,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na


#### **tissue_type**

In [150]:
adata.obs['tissue_type'] = ['tissue'] * len(adata.obs)

In [151]:
adata.obs['tissue_type'] = adata.obs['tissue_type'].astype('category')

#### **tissue_ontology_term_id**

In [152]:
adata.obs['tissue_ontology_term_id'] = ['UBERON:0015476'] * len(adata.obs)

In [153]:
adata.obs['tissue_ontology_term_id'] = adata.obs['tissue_ontology_term_id'].astype('category')

In [154]:
list(adata.obs['tissue_ontology_term_id'].unique())

['UBERON:0015476']

In [155]:
# view obs

In [156]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type,tissue_type,tissue_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,8420,54784,1315,2.400336,13115,23.939470,AAACAGCTTTCAGAAG-1,...,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0015476
AAACCGGGTAGGTACC-1,1,42,28,1955,4347,44,1.012192,443,10.190936,AAACCGGGTAGGTACC-1,...,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0015476
AAACCTCATGAAGTTG-1,1,37,19,8114,47734,590,1.236016,10006,20.961998,AAACCTCATGAAGTTG-1,...,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0015476
AAACGAGACGGTTGAT-1,1,35,79,6880,23861,617,2.585809,4262,17.861784,AAACGAGACGGTTGAT-1,...,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0015476
AAACTTGCAAACGTAT-1,1,45,19,7110,30728,764,2.486332,4286,13.948191,AAACTTGCAAACGTAT-1,...,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0015476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTAATCCGTACTCG-1,1,35,55,2856,10697,104,0.972235,949,8.871646,TTGTAATCCGTACTCG-1,...,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0015476
TTGTGAGGCATGACGC-1,1,43,13,5261,16840,185,1.098575,2632,15.629454,TTGTGAGGCATGACGC-1,...,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0015476
TTGTGTTTCCCGAAAG-1,1,51,59,5797,25722,242,0.940829,2508,9.750408,TTGTGTTTCCCGAAAG-1,...,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0015476


In [157]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'max_c2l_column', 'max_c2l_column_value',
       'cell_type_ontology_term_id', 'donor_id',
       'development_stage_ontology_term_id', 'disease_ontology_term_id',
       'is_primary_data', 

In [158]:
del adata.obs['barcode']
del adata.obs['max_c2l_column']
del adata.obs['max_c2l_column_value']
del adata.obs['array_row']
del adata.obs['array_col']

#### **obsm (Embeddings)**

In [159]:
adata.obsm

AxisArrays with keys: X_spatial

In [160]:
adata.obsm.keys()

KeysView(AxisArrays with keys: X_spatial)

#### **uns (Dataset Metadata)**

In [161]:
adata.uns

OverloadedDict, wrapping:
	{'leiden': {'params': {'n_iterations': -1, 'random_state': 0, 'resolution': 1}}, 'log1p': {}, 'neighbors': {'connectivities_key': 'connectivities', 'distances_key': 'distances', 'params': {'method': 'umap', 'metric': 'euclidean', 'n_neighbors': 15, 'random_state': 0}}, 'pca': {'params': {'use_highly_variable': False, 'zero_center': True}, 'variance': array([584.367   , 340.0391  , 166.46225 , 106.88909 ,  97.35766 ,
        77.88227 ,  60.50748 ,  51.8118  ,  49.71711 ,  48.353783,
        46.35967 ,  45.1817  ,  44.398014,  43.654167,  42.952454,
        42.617794,  41.855984,  41.476337,  40.936016,  40.094334,
        39.20836 ,  38.16314 ,  37.88483 ,  37.634743,  36.881374,
        35.830788,  35.13198 ,  34.726864,  34.41222 ,  33.847767,
        33.662617,  33.179363,  33.118484,  33.04452 ,  32.591328,
        32.345726,  32.14421 ,  31.92326 ,  31.795712,  31.68235 ,
        31.54376 ,  31.268969,  31.186768,  31.085997,  30.969942,
        30.708342

In [162]:
adata.uns['image_caption'] = 'Shown here is an image of 10 μm thick slice of the skin from the nose stained with H&E'

In [163]:
adata.uns['title'] = 'Visium spatial - bcc_face_nose_A'

In [164]:
adata.uns['default_embedding'] = 'X_spatial'

In [165]:
adata.uns.keys()

dict_keys(['leiden', 'log1p', 'neighbors', 'pca', 'spatial', 'umap', 'image_caption', 'title', 'default_embedding'])

### **Final checks and adjustments**

In [166]:
adata

AnnData object with n_obs × n_vars = 1200 × 36390
    obs: 'in_tissue', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes', 'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc', 'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes', 'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts', 'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC', 'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes', 'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1', 'assay_ontology_term_id', 'cell_type_ontology_term_id', 'donor_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'is_primary_data', 'organism_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'sex_ontology_term_id', 'suspension_type', 'tissue_t

In [167]:
adata.obs.dtypes

in_tissue                                      int64
n_genes_by_counts                              int32
total_counts                                   int32
total_counts_mt                                int32
pct_counts_mt                                float32
total_counts_rb                                int32
pct_counts_rb                                float32
c2l_Th                                       float64
c2l_NK                                       float64
c2l_APOD+ fibroblasts                        float64
c2l_CD8+ T RM                                float64
c2l_T reg                                    float64
c2l_Macro1_2                                 float64
c2l_DC1                                      float64
c2l_SFRP2+ fibroblasts                       float64
c2l_TAGLN+ pericytes                         float64
c2l_POSTN+ fibroblasts                       float64
c2l_RGS5+ pericytes                          float64
c2l_VEC                                      f

In [168]:
dty = pd.DataFrame(adata.var.dtypes, columns = ['dtype'])
for c in dty[dty['dtype'] == 'float64'].index.values:
    adata.var[c] = adata.var[c].astype('float32')
    print(f"changed {c} from float64 to float32")
for c in dty[dty['dtype'] == 'int64'].index.values:
    adata.var[c] = adata.var[c].astype('int32') 
    print(f"changed {c} from int64 to int32")

In [169]:
dty = pd.DataFrame(adata.obs.dtypes, columns = ['dtype'])
for c in dty[dty['dtype'] == 'float64'].index.values:
    adata.obs[c] = adata.obs[c].astype('float32')
    print(f"changed {c} from float64 to float32")
for c in dty[dty['dtype'] == 'int64'].index.values:
    adata.obs[c] = adata.obs[c].astype('int32') 
    print(f"changed {c} from int64 to int32")
for c in dty[dty['dtype'] == 'object'].index.values:
    adata.obs[c] = adata.obs[c].astype('category') 
    print(f"changed {c} from object to category")

changed c2l_Th from float64 to float32
changed c2l_NK from float64 to float32
changed c2l_APOD+ fibroblasts from float64 to float32
changed c2l_CD8+ T RM from float64 to float32
changed c2l_T reg from float64 to float32
changed c2l_Macro1_2 from float64 to float32
changed c2l_DC1 from float64 to float32
changed c2l_SFRP2+ fibroblasts from float64 to float32
changed c2l_TAGLN+ pericytes from float64 to float32
changed c2l_POSTN+ fibroblasts from float64 to float32
changed c2l_RGS5+ pericytes from float64 to float32
changed c2l_VEC from float64 to float32
changed c2l_Tc from float64 to float32
changed c2l_ILC_NK from float64 to float32
changed c2l_BC from float64 to float32
changed c2l_Monocytes from float64 to float32
changed c2l_MastC from float64 to float32
changed c2l_Melanocytes from float64 to float32
changed c2l_DC2 from float64 to float32
changed c2l_LEC from float64 to float32
changed c2l_PlasmaC from float64 to float32
changed c2l_PTGDS+ fibroblasts from float64 to float32
chan

In [170]:
adata.obs

,in_tissue,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,c2l_Th,c2l_NK,c2l_APOD+ fibroblasts,...,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type,tissue_type,tissue_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,8420,54784,1315,2.400336,13115,23.939470,0.017032,0.001726,0.000117,...,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0015476
AAACCGGGTAGGTACC-1,1,1955,4347,44,1.012192,443,10.190936,0.007060,0.272603,0.053994,...,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0015476
AAACCTCATGAAGTTG-1,1,8114,47734,590,1.236016,10006,20.961998,0.027300,0.049147,0.000904,...,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0015476
AAACGAGACGGTTGAT-1,1,6880,23861,617,2.585809,4262,17.861784,0.001880,0.001269,0.000093,...,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0015476
AAACTTGCAAACGTAT-1,1,7110,30728,764,2.486332,4286,13.948191,0.019222,0.322737,0.004532,...,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0015476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTAATCCGTACTCG-1,1,2856,10697,104,0.972235,949,8.871646,0.016169,0.340894,0.123071,...,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0015476
TTGTGAGGCATGACGC-1,1,5261,16840,185,1.098575,2632,15.629454,0.012367,0.196257,0.008664,...,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0015476
TTGTGTTTCCCGAAAG-1,1,5797,25722,242,0.940829,2508,9.750408,0.008640,0.689163,0.039709,...,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0015476


In [171]:
adata.obs.columns

Index(['in_tissue', 'n_genes_by_counts', 'total_counts', 'total_counts_mt',
       'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'c2l_Th', 'c2l_NK',
       'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2',
       'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'cell_type_ontology_term_id', 'donor_id',
       'development_stage_ontology_term_id', 'disease_ontology_term_id',
       'is_primary_data', 'organism_ontology_term_id',
       'self_reported_ethnicity_ontology_term_id', 'sex_ontology

In [172]:
adata.var

,feature_is_filtered
ENSG00000243485,True
ENSG00000237613,True
ENSG00000186092,True
ENSG00000238009,False
ENSG00000239945,True
...,...
ENSG00000277836,True
ENSG00000278633,True
ENSG00000276017,True
ENSG00000278817,False


In [173]:
adata.obs

,in_tissue,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,c2l_Th,c2l_NK,c2l_APOD+ fibroblasts,...,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type,tissue_type,tissue_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,8420,54784,1315,2.400336,13115,23.939470,0.017032,0.001726,0.000117,...,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0015476
AAACCGGGTAGGTACC-1,1,1955,4347,44,1.012192,443,10.190936,0.007060,0.272603,0.053994,...,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0015476
AAACCTCATGAAGTTG-1,1,8114,47734,590,1.236016,10006,20.961998,0.027300,0.049147,0.000904,...,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0015476
AAACGAGACGGTTGAT-1,1,6880,23861,617,2.585809,4262,17.861784,0.001880,0.001269,0.000093,...,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0015476
AAACTTGCAAACGTAT-1,1,7110,30728,764,2.486332,4286,13.948191,0.019222,0.322737,0.004532,...,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0015476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTAATCCGTACTCG-1,1,2856,10697,104,0.972235,949,8.871646,0.016169,0.340894,0.123071,...,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0015476
TTGTGAGGCATGACGC-1,1,5261,16840,185,1.098575,2632,15.629454,0.012367,0.196257,0.008664,...,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0015476
TTGTGTTTCCCGAAAG-1,1,5797,25722,242,0.940829,2508,9.750408,0.008640,0.689163,0.039709,...,bcc_face_nose_A,HsapDv:0000217,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0015476


In [174]:
adata.obs.columns

Index(['in_tissue', 'n_genes_by_counts', 'total_counts', 'total_counts_mt',
       'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'c2l_Th', 'c2l_NK',
       'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2',
       'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'cell_type_ontology_term_id', 'donor_id',
       'development_stage_ontology_term_id', 'disease_ontology_term_id',
       'is_primary_data', 'organism_ontology_term_id',
       'self_reported_ethnicity_ontology_term_id', 'sex_ontology

In [175]:
#check the format of expression matrix

In [176]:
adata.X

<1200x36390 sparse matrix of type '<class 'numpy.float32'>'
	with 23416800 stored elements in Compressed Sparse Row format>

In [177]:
araw.X

<1200x36390 sparse matrix of type '<class 'numpy.float32'>'
	with 5557613 stored elements in Compressed Sparse Row format>

In [178]:
#Copy raw counts to adata.raw

In [179]:
adata.raw = araw

In [180]:
del adata.uns['log1p']

In [181]:
#write the curated object to final_objects folder

In [182]:
adata.write('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/spatial-skin/Final_objects/bcc_face_nose1.h5ad', compression = 'gzip')